# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [18]:
# Dependencies and Setup
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [109]:
cities_data_path = './output_data/cities.csv'
cities_df = pd.read_csv(cities_data_path)
cities_df.head()
# cities_df.count()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Mahajanga,20,MG,1585698666,88,-15.72,46.32,78.800,2.236940
1,Lloydminster,20,CA,1585698435,60,53.28,-110.00,10.400,16.105968
2,Salalah,40,OM,1585698667,88,17.02,54.09,80.600,1.118470
3,Dikson,100,RU,1585698667,93,73.51,80.55,-3.316,4.294925
4,Seoul,20,KR,1585697999,34,37.57,126.98,51.800,4.227817


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [110]:
gmaps.configure(api_key=g_key) # Fill in with your API key

locations = cities_df[['Lat', 'Lng']]
weights = cities_df['Humidity']

In [111]:
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=50,
                                 point_radius = 2, opacity=0.5))
plt.savefig('./output_data/vacation_heatmap.png')
fig

<IPython.core.display.Javascript object>

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [112]:
vac_cities_df = cities_df.copy()
vac_cities_df = vac_cities_df.loc[(vac_cities_df['Max Temp'] > 70) & 
                                  (vac_cities_df['Max Temp'] < 80) & 
                                  (vac_cities_df['Wind Speed'] < 10) &
                                  (vac_cities_df['Cloudiness'] == 0)]

vac_cities_df.count()

City          7
Cloudiness    7
Country       7
Date          7
Humidity      7
Lat           7
Lng           7
Max Temp      7
Wind Speed    7
dtype: int64

In [113]:
vac_cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
90,Āsind,0,IN,1585698693,27,25.73,74.33,71.888,7.493749
91,Inhambane,0,MZ,1585698693,71,-23.86,35.38,73.274,6.934514
312,Abu Dhabi,0,AE,1585698565,68,24.47,54.37,75.002,2.236940
346,Inverell,0,AU,1585698757,59,-29.78,151.12,77.000,1.990877
429,Narasannapeta,0,IN,1585698776,72,18.42,84.05,79.736,4.787052
449,Karrāpur,0,IN,1585698781,22,23.95,78.85,74.426,4.429141
459,Turbat,0,PK,1585698783,59,26.00,63.04,71.672,2.527742


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [114]:
hotel_df = vac_cities_df.copy()

In [115]:
for index, data in hotel_df.iterrows():
    print(index, data['Lat'], data['Lng'])
    latitude = data['Lat']
    longitude = data['Lng']
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    hotel_name = response['results'][7]['formatted_address']
    hotel_df.loc[index, 'Hotel Name'] = hotel_name
    print(hotel_name)
    

90 25.73 74.33
Panchayat Samiti Rd, Ward Number 19, Asind, Rajasthan 311301, India
91 -23.86 35.38
centro de negócios, Rua Vladimir Lenine, R. Da O.U.A, Inhambane, Mozambique
312 24.47 54.37
Al Salam St - Abu Dhabi - United Arab Emirates
346 -29.78 151.12
49-51 Byron St, Inverell NSW 2360, Australia
429 18.42 84.05
#23-333, Palleipeta Road, Narasannapet, Srikakulam, Andhra Pradesh 532421, India
449 23.95 78.85
Simariya, Madhya Pradesh 470337, India
459 26.0 63.04
Shop No 104, Salam Market, Main Rd, Turbat, Kech, Balochistan 92600, Pakistan


In [116]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
90,Āsind,0,IN,1585698693,27,25.73,74.33,71.888,7.493749,"Panchayat Samiti Rd, Ward Number 19, Asind, Ra..."
91,Inhambane,0,MZ,1585698693,71,-23.86,35.38,73.274,6.934514,"centro de negócios, Rua Vladimir Lenine, R. Da..."
312,Abu Dhabi,0,AE,1585698565,68,24.47,54.37,75.002,2.236940,Al Salam St - Abu Dhabi - United Arab Emirates
346,Inverell,0,AU,1585698757,59,-29.78,151.12,77.000,1.990877,"49-51 Byron St, Inverell NSW 2360, Australia"
429,Narasannapeta,0,IN,1585698776,72,18.42,84.05,79.736,4.787052,"#23-333, Palleipeta Road, Narasannapet, Srikak..."
449,Karrāpur,0,IN,1585698781,22,23.95,78.85,74.426,4.429141,"Simariya, Madhya Pradesh 470337, India"
459,Turbat,0,PK,1585698783,59,26.00,63.04,71.672,2.527742,"Shop No 104, Salam Market, Main Rd, Turbat, Ke..."


In [117]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Lat", "Lng"]]

In [135]:
# Add marker layer ontop of heat map
# info =  ['<p>Atucha</p>', 'Embalse', 'Armenia', 'BR', 'Doel', 'aa', 'bb']
hover_text = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights, dissipating=False, max_intensity=50,
                                 point_radius = 2, opacity=1))
fig.add_layer(gmaps.marker_layer(locations2, hover_text=hover_text, label='', info_box_content='TEST'))
fig

# Display Map


Figure(layout=FigureLayout(height='420px'))